# 1.원소별 연산 (element-wise operation)

: 텐서에 있는 각 원소에 독립적으로 적용

In [1]:
import numpy as np

## Relu 간단한 구현

In [2]:
def naive_relu(x):
    assert x.ndim==2 # x는 2D numpy 배열
    
    x=x.copy()       # 입력 텐서 자체를 바꾸지 않도록 복사 
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j]=max(x[i,j],0)
            
    return x

In [3]:
a=np.array([[-1,0,1],[1,2,-3]])
naive_relu(a)

array([[0, 0, 1],
       [1, 2, 0]])

## 덧셈 간단한 구현 

In [4]:
def naive_add(x,y):
    assert len(x.shape)==2     # x는 2D numpy 배열
    assert x.shape==y.shape    # x와 y의 shape 같아야함
    
    x=x.copy()                 # 입력 텐서 자체를 바꾸지 않도록 복사 
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j]+=y[i,j]
    return x

In [5]:
a=np.array([[-1,0,1],[1,2,-3]])
b=np.array([[2,1,-2],[3,1,-3]])

naive_add(a,b)

array([[ 1,  1, -1],
       [ 4,  3, -6]])

위에서 copy를 빼면 어떻게 될까?

In [6]:
a=np.array([1,2,3])

# copy 해줌
def add_one(x):
    
    x=x.copy()
    
    x+=1
    
    return x
# copy 안해줌
def add_one2(x):
    
    x+=1
    
    return x

print("a:",a)
print("add_one:",add_one(a))
print("a:",a)

print()

print("a:",a)
print("add_one2:",add_one2(a)) # copy 안해주면 원래 값 달라짐
print("a:",a)

a: [1 2 3]
add_one: [2 3 4]
a: [1 2 3]

a: [1 2 3]
add_one2: [2 3 4]
a: [2 3 4]


In [7]:
a=np.array([1,2,3])
x=a.copy()               # numpy의 copy는 deepcopy인가봄
x[0]+=1
print("a:",a)
print("x:",x)

a: [1 2 3]
x: [2 2 3]


## Relu와 덧셈 numpy 내장 함수 이용하기 (엄청 빠름)

In [8]:
x=np.array([1,0,1])
y=np.array([-1,-1,3])


z=x+y
print("덧셈:",z)

z=np.maximum(z,0.)
print("relu:",z)

덧셈: [ 0 -1  4]
relu: [0. 0. 4.]


## 원소별 곱셈

\* 연산자 이용한다

In [9]:
a=np.array([[-1,0,1],[1,2,-3]])
b=np.array([[-1,3,3],[-2,2,-3]])

a*b

array([[ 1,  0,  3],
       [-2,  4,  9]])

# 2.브로드캐스팅 (broadcasting)

: 모호하지 않고 실행가능하다면 작은 텐서가 큰 텐서의 크기에 맞추어 __'브로드캐스팅'__ 된다

브로드캐스팅의 2가지 단계

1. 큰 텐서의 ndim에 맞도록 작은 텐서에 축 추가

2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복

작은 텐서를 큰 텐서와 같은 크기로 만들어주는 것이 아니라,       (메모리 낭비)

반복작업을 한다!

## 브로드캐스팅 단순한 구현

In [10]:
def naive_add_matrix_and_vector(x,y):
    assert len(x.shape)==2                 # x는 2D numpy 배열
    assert len(y.shape)==1                 # y는 numpy 벡터
    assert x.shape[1]==y.shape[0]          # x의 col 개수와 y의 길이 같아야함
    
    x=x.copy()
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j]+=y[j]
            
    return x

In [11]:
a=np.array([[-1,0,1],[1,2,-3]])
b=np.array([2,2,3])

naive_add_matrix_and_vector(a,b)

array([[1, 2, 4],
       [3, 4, 0]])

# 3.텐서 곱셈 (내적)

: not 원소별 곱셈 but 내적 (dot product)

\* 벡터의 dot product 결과는 scalar

## numpy 내장 함수 dot 사용


``` numpy.dot(arr1,arr2) ```

arr1과 arr2를 내적한 numpy 벡터를 리턴

In [12]:
a=np.array([-1,0,1])
b=np.array([-1,3,3])

np.dot(a,b)

4

## 벡터의 내적 간단한 구현

In [13]:
def naive_vector_dot(x,y):
    assert len(x.shape)==1          # x는 numpy 벡터
    assert len(y.shape)==1          # y는 numpy 벡터
    assert x.shape[0]==y.shape[0]   # x,y벡터의 차원 같다
    
    z=0.
    for i in range(x.shape[0]):
        z+=x[i]*y[i]
        
    return z

In [14]:
naive_vector_dot(a,b)

4.0

## 행렬과 벡터 사이의 곱셈 간단한 구현

$A$: mxn 배열

$x$: nx1 벡터

$ \Rightarrow A \cdot x $ : mx1 벡터

In [15]:
def naive_matrix_vector_dot(x,y):
    assert len(x.shape)==2               # x는 2D numpy 배열
    assert len(y.shape)==1               # y는 numpy 벡터
    assert x.shape[1]==y.shape[0]        # x의 col 개수와 y의 차원이 같아야함
    
    # 결과인 z는 차원이 x의 row 개수인 벡터
    z=np.zeros(x.shape[0])
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i]+=x[i,j]*y[j]
            
    return z

이전 구현한 함수 이용하기

In [16]:
def naive_matrix_vector_dot2(x,y):
    
    z=np.zeros(x.shape[0])
    
    for i in range(x.shape[0]):
        z[i]=naive_vector_dot(x[i,:],y)
    
    return z

In [17]:
a=np.array([[-1,0,1],[1,2,-3]])
b=np.array([2,2,3])

print("첫 번쨰 방법 :",naive_matrix_vector_dot(a,b))
print("두 번쨰 방법 :",naive_matrix_vector_dot2(a,b))

첫 번쨰 방법 : [ 1. -3.]
두 번쨰 방법 : [ 1. -3.]


## 행렬의 곱셈 간단한 구현

In [18]:
def naive_matrix_dot(x,y):
    assert len(x.shape)==2
    assert len(y.shape)==2
    assert x.shape[1]==y.shape[0]
    
    z=np.zeros((x.shape[0],y.shape[1]))
    
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):        
            row_x=x[i,:]
            col_y=y[:,j]
            z[i,j]=naive_vector_dot(row_x,col_y)
    
    return z

In [19]:
a=np.array([[-1,0,1],[1,2,-3]])
b=np.array([[1,1,1],[2,2,2],[3,3,3]])

naive_matrix_dot(a,b)

array([[ 2.,  2.,  2.],
       [-4., -4., -4.]])

# 4.텐서 크기 변환 (tensor reshaping)

: 특정 크기에 맞게 열과 행을 재배열

``` arr.reshape((m,n)) ```

arr의 shape가 (m,n)이 되도록 재배열

In [20]:
x=np.array([[0,1],[2,3],[4,5]])
x.shape

(3, 2)

In [21]:
x=x.reshape((6,1))
print(x)
print("x의 shape:",x.shape)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]]
x의 shape: (6, 1)


In [22]:
x=x.reshape((2,3))
print(x)
print("x의 shape:",x.shape)

[[0 1 2]
 [3 4 5]]
x의 shape: (2, 3)


## 전치 (transposition)

: 행과 열을 바꾸는 것

``` numpy.transpose(arr)```

arr를 전치시킨 numpy 배열을 리턴

In [23]:
print(x)
print("x의 shape:",x.shape)

[[0 1 2]
 [3 4 5]]
x의 shape: (2, 3)


In [24]:
x=np.transpose(x)
print(x)
print("x의 shape:",x.shape)

[[0 3]
 [1 4]
 [2 5]]
x의 shape: (3, 2)
